# Import

In [1]:
from keras.models import load_model
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import keras
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
import json
from scipy.optimize import minimize
from datetime import datetime, timedelta
import math

# Download stock data

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # write column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        #for symbol in stock_symbols:
            # load share data for the symbol (Share name) and the given time period
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
        stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei

In [3]:
# Enter symbols
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']
start_date = '2014-01-01'
end_date = '2018-12-31'

# load and store the data for every share
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # read CSV and select select the desired columns
    df = pd.read_csv(output_file, usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Search CSV file for empty lines and remove them
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # filter the empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the adjusted rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
# store the data frames for different symbols in the dictionary
dfPre = {}

for symbol in stock_symbols:
    # load CSV
    dfPre[symbol] = pd.read_csv(f'stock_data_{symbol}.csv')

    # Drop empty rows
    dfPre[symbol].dropna(inplace=True)

    # transform date to index
    dfPre[symbol].set_index('Date', inplace=True)

    # Check the number of rows
    num_rows = dfPre[symbol].shape[0]

In [5]:
# Check missing values
missing_values_beforePre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# missing values are replaced with teh mean of the row before and after the actual row
for symbol in stock_symbols:
    for column in dfPre[symbol].columns:
        missing_valuesPre = dfPre[symbol][column].isnull()
        dfPre[symbol].loc[missing_valuesPre, column] = (dfPre[symbol][column].shift() + dfPre[symbol][column].shift(-1)) / 2

# Check if there are still any missing values 
missing_values_afterPre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# Output of the missing values before and after the Treatment
for symbol in stock_symbols:
    print(f"Fehlende Werte vor der Behandlung für {symbol} gefunden:", missing_values_beforePre[symbol])
    print(f"Fehlende Werte nach der Behandlung für {symbol} gefunden:", missing_values_afterPre[symbol])

# Output of the length of the data frame for all symbols
for symbol in stock_symbols:
    print(f"Länge des Datensatzes für {symbol}:", len(dfPre[symbol]))

Fehlende Werte vor der Behandlung für ALV.DE gefunden: False
Fehlende Werte nach der Behandlung für ALV.DE gefunden: False
Fehlende Werte vor der Behandlung für DBK.DE gefunden: False
Fehlende Werte nach der Behandlung für DBK.DE gefunden: False
Fehlende Werte vor der Behandlung für VOW3.DE gefunden: False
Fehlende Werte nach der Behandlung für VOW3.DE gefunden: False
Fehlende Werte vor der Behandlung für BMW.DE gefunden: False
Fehlende Werte nach der Behandlung für BMW.DE gefunden: False
Fehlende Werte vor der Behandlung für ADS.DE gefunden: False
Fehlende Werte nach der Behandlung für ADS.DE gefunden: False
Fehlende Werte vor der Behandlung für BEI.DE gefunden: False
Fehlende Werte nach der Behandlung für BEI.DE gefunden: False
Fehlende Werte vor der Behandlung für DTE.SG gefunden: False
Fehlende Werte nach der Behandlung für DTE.SG gefunden: False
Fehlende Werte vor der Behandlung für SAP.DE gefunden: False
Fehlende Werte nach der Behandlung für SAP.DE gefunden: False
Fehlende Werte

# Read close values at the begin of the time period

In [6]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = end_date_obj

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2022-12-25
2022-12-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)

print(combined_data_begin)

     Symbol  Close_Value
0    ALV.DE   200.899994
1    DBK.DE    10.588000
2   VOW3.DE   116.419998
3    BMW.DE    83.379997
4    ADS.DE   127.459999
5    BEI.DE   107.199997
6    DTE.SG    18.700001
7    SAP.DE    96.389999
8   1COV.DE    36.549999
9    BAS.DE    46.389999
10  EOAN.DE     9.334000
11   RWE.DE    41.590000


# load share value from 2019

In [8]:
start_date = '2019-12-01'
end_date = '2019-12-31'
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

Startdatum: 2023-12-01, Enddatum: 2023-12-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])
# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

241.9499969482422
Liste der letzten Close-Werte:
ALV.DE: 241.9499969482422
DBK.DE: 12.36400032043457
VOW3.DE: 111.8000030517578
BMW.DE: 100.77999877929688
ADS.DE: 184.16000366210935
BEI.DE: 135.6999969482422
DTE.SG: 21.690000534057617
SAP.DE: 139.47999572753906
1COV.DE: 52.68000030517578
BAS.DE: 48.77999877929688
EOAN.DE: 12.149999618530272
RWE.DE: 41.18000030517578


# Split into blocks of 20 (20 business days per month)

In [10]:
# Dictionary for the splitted data
df_split = {}

# Quantity of the blocks per symbol
block_size = 252

for symbol, df in dfPre.items():
    # Quantity of the rows in the data frame
    num_rows = len(df)

    # Split the data frame in to blocks
    split_dfs = np.array_split(df, num_rows // block_size)

    # Add the solited blocks to the dictionary
    df_split[symbol] = split_dfs

# Output result
for symbol, split_dfs in df_split.items():
    print(f"Symbol: {symbol}")
    for i, df_block in enumerate(split_dfs):
        print(f"Block {i+1}:")
        print(df_block)
        print()

Symbol: ALV.DE
Block 1:
                  Open        High         Low       Close   Volume
Date                                                               
2014-01-02  130.949997  131.399994  127.849998  128.149994  1524683
2014-01-03  128.000000  128.449997  127.150002  127.900002   853713
2014-01-06  127.650002  128.800003  127.599998  128.399994   900878
2014-01-07  128.350006  130.350006  128.149994  130.100006  1189652
2014-01-08  130.149994  130.149994  128.850006  129.399994  1214204
...                ...         ...         ...         ...      ...
2014-12-23  139.500000  140.000000  138.899994  139.699997  1040720
2014-12-29  139.000000  139.399994  137.050003  139.250000  1083568
2014-12-30  138.149994  138.300003  137.350006  137.350006   695403
2015-01-02  137.300003  137.850006  135.750000  136.699997  1494235
2015-01-05  136.500000  137.500000  133.149994  133.350006  2091329

[254 rows x 5 columns]

Block 2:
                  Open        High         Low       Close

                 Open       High        Low      Close   Volume
Date                                                           
2014-01-02  78.400002  78.720001  76.260002  76.379997  2960768
2014-01-03  76.389999  76.849998  76.029999  76.570000  1707214
2014-01-06  76.209999  76.989998  76.059998  76.419998  1749064
2014-01-07  78.430000  78.860001  78.209999  78.519997  3974187
2014-01-08  77.989998  78.419998  77.400002  78.000000  2642750
...               ...        ...        ...        ...      ...
2014-12-23  70.419998  71.010002  69.959999  71.010002  1754831
2014-12-29  71.099998  71.099998  69.739998  70.900002  1515022
2014-12-30  70.599998  70.730003  69.879997  69.879997  1320390
2015-01-02  70.300003  70.690002  68.800003  69.690002  2260387
2015-01-05  69.430000  69.860001  66.580002  66.709999  4836613

[254 rows x 5 columns]

Block 2:
                 Open       High        Low      Close   Volume
Date                                                           
2015-0

# Calculate the return for every month

In [11]:
# Dictionary for the increase/decrease per symbol
change_per_symbol = {}

for symbol, split_dfs in df_split.items():
    changes = []

    # Iterate over every block of the current symbol
    for df_block in split_dfs:
        # Extract the first and the lat close value of the current block
        first_close = df_block.iloc[0]['Close']
        last_close = df_block.iloc[-1]['Close']

        # Calculate the change of the close value
        change = (last_close - first_close) / first_close * 100  # in Prozent
        changes.append(change)

    # Save the Change for the current symbol
    change_per_symbol[symbol] = changes

# Output result
for symbol, changes in change_per_symbol.items():
    print(f"Symbol: {symbol}")
    for i, change in enumerate(changes):
        print(f"Block {i+1}: {change:.2f}%")
    print()


Symbol: ALV.DE
Block 1: 4.06%
Block 2: 17.23%
Block 3: 0.19%
Block 4: 20.03%
Block 5: -10.14%
Block 6: 23.08%
Block 7: -6.77%
Block 8: 8.21%
Block 9: -9.73%

Symbol: DBK.DE
Block 1: -25.49%
Block 2: -8.86%
Block 3: -18.83%
Block 4: 0.41%
Block 5: -55.52%
Block 6: 3.35%
Block 7: 23.37%
Block 8: 27.59%
Block 9: -14.38%

Symbol: VOW3.DE
Block 1: -12.07%
Block 2: -31.57%
Block 3: 15.85%
Block 4: 18.51%
Block 5: -18.67%
Block 6: 25.87%
Block 7: -19.40%
Block 8: 30.50%
Block 9: -37.99%

Symbol: BMW.DE
Block 1: 1.84%
Block 2: 6.98%
Block 3: 1.35%
Block 4: -4.40%
Block 5: -20.08%
Block 6: 3.77%
Block 7: -5.20%
Block 8: 38.85%
Block 9: -12.78%

Symbol: ADS.DE
Block 1: -40.10%
Block 2: 59.75%
Block 3: 75.46%
Block 4: 11.99%
Block 5: 10.05%
Block 6: 58.52%
Block 7: -4.81%
Block 8: -13.41%
Block 9: -48.49%

Symbol: BEI.DE
Block 1: -9.72%
Block 2: 23.81%
Block 3: 0.49%
Block 4: 20.70%
Block 5: -5.03%
Block 6: 15.21%
Block 7: -12.20%
Block 8: -4.45%
Block 9: 19.83%

Symbol: DTE.SG
Block 1: 9.06%
Blo

# Calculate the mean return over the period of 4 years

In [12]:
# Dictionary for the mean return per symbol
average_return_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    # Calculate the mean return for the current symbol
    average_return = sum(changes) / len(changes)
    average_return_per_symbol[symbol] = average_return

# Output result
for symbol, avg_return in average_return_per_symbol.items():
    print(f"Durchschnittliche Rendite für {symbol}: {avg_return:.2f}%")
    
# Convert mean return to DataFrame
average_return_per_symbol_df = pd.DataFrame(list(average_return_per_symbol.items()), columns=['Symbol', 'Average_Return'])
print(average_return_per_symbol_df)


Durchschnittliche Rendite für ALV.DE: 5.13%
Durchschnittliche Rendite für DBK.DE: -7.60%
Durchschnittliche Rendite für VOW3.DE: -3.22%
Durchschnittliche Rendite für BMW.DE: 1.15%
Durchschnittliche Rendite für ADS.DE: 12.11%
Durchschnittliche Rendite für BEI.DE: 5.40%
Durchschnittliche Rendite für DTE.SG: 5.79%
Durchschnittliche Rendite für SAP.DE: 6.78%
Durchschnittliche Rendite für 1COV.DE: 11.94%
Durchschnittliche Rendite für BAS.DE: -3.59%
Durchschnittliche Rendite für EOAN.DE: 0.03%
Durchschnittliche Rendite für RWE.DE: 10.20%
     Symbol  Average_Return
0    ALV.DE        5.128629
1    DBK.DE       -7.597787
2   VOW3.DE       -3.218171
3    BMW.DE        1.145897
4    ADS.DE       12.107460
5    BEI.DE        5.403940
6    DTE.SG        5.793720
7    SAP.DE        6.784564
8   1COV.DE       11.943886
9    BAS.DE       -3.587521
10  EOAN.DE        0.027457
11   RWE.DE       10.195432


# Accuracy of the mean return to the actual return

In [13]:
MonetarizedMeanReturn = {}
mse_per_symbol = {}
# list to store the actual and predicted value for every symbol
all_actual_values = []
all_predicted_values = []
for symbol in stock_symbols:
    # Get the close value for the current symbol
    value_begin = combined_data_begin.loc[combined_data_begin['Symbol'] == symbol, 'Close_Value'].values
    # Get the close value for the current symbol from last_close_values
    value_end = next(value for sym, value in last_close_values if sym == symbol)
    # Get the average return for the current symbol
    average_return = average_return_per_symbol_df.loc[average_return_per_symbol_df['Symbol'] == symbol, 'Average_Return'].values
    MonetarizedMeanReturn[symbol] = value_begin * (1 + (average_return/100))
    mse = mean_squared_error([value_end], [MonetarizedMeanReturn[symbol]])
    #Print(mse)
    
    #dd mse to the list
    mse_per_symbol[symbol] = mse
    
    # Add the actual and predicted value to the main list
    all_actual_values.append(value_end)
    all_predicted_values.append(MonetarizedMeanReturn[symbol])
    
total_mse = mean_squared_error(all_actual_values, all_predicted_values)
standard_deviation = math.sqrt(total_mse)
standard_deviation_share = standard_deviation / 12

# Output of the mse for all values
print(f"Gesamter Mean Squared Error für alle Vorhersagen: {total_mse}")
print(f"Gesamte Standardabweichung für alle Vorhersagen: {standard_deviation}")
print(f"Gesamte Standardabweichung pro Aktie: {standard_deviation_share}")

Gesamter Mean Squared Error für alle Vorhersagen: 413.81526806606917
Gesamte Standardabweichung für alle Vorhersagen: 20.3424499032459
Gesamte Standardabweichung pro Aktie: 1.695204158603825


# Standard deviation

In [14]:
# Dictionary for the standard deviation per symbol
standard_deviation_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    
    # Calculate the difference to the power of 2
    squared_diff = [(change - average_return_per_symbol[symbol]) ** 2 for change in changes]
    
    # Sum up the differences
    sum_squared_diff = sum(squared_diff)

    # Calculate the standard deviation
    standard_deviation = np.sqrt(sum_squared_diff / (len(changes) - 1))
    
    # Save the standard deviation in the dictionary
    standard_deviation_per_symbol[symbol] = standard_deviation

# Output result
for symbol, std_dev in standard_deviation_per_symbol.items():
    print(f"Standardabweichung für {symbol}: {std_dev:.2f}")
    
# Convert mean return to DataFrame
standard_deviation_per_symbol_df = pd.DataFrame(list(standard_deviation_per_symbol.items()), columns=['Symbol', 'Standard_Deviation'])
print(standard_deviation_per_symbol_df)

Standardabweichung für ALV.DE: 12.85
Standardabweichung für DBK.DE: 25.42
Standardabweichung für VOW3.DE: 26.01
Standardabweichung für BMW.DE: 16.50
Standardabweichung für ADS.DE: 44.41
Standardabweichung für BEI.DE: 14.34
Standardabweichung für DTE.SG: 10.99
Standardabweichung für SAP.DE: 20.80
Standardabweichung für 1COV.DE: 48.27
Standardabweichung für BAS.DE: 20.42
Standardabweichung für EOAN.DE: 22.65
Standardabweichung für RWE.DE: 30.63
     Symbol  Standard_Deviation
0    ALV.DE           12.851295
1    DBK.DE           25.424479
2   VOW3.DE           26.007048
3    BMW.DE           16.501465
4    ADS.DE           44.408236
5    BEI.DE           14.343655
6    DTE.SG           10.991185
7    SAP.DE           20.800510
8   1COV.DE           48.267332
9    BAS.DE           20.422438
10  EOAN.DE           22.652058
11   RWE.DE           30.633122


# Historic data from 2018 for the covarianz matrix

In [15]:
# Definition of the time period
start_date2018 = '2018-01-01'
end_date2018 = '2018-12-31'
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2022-12-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file

In [16]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)
    
# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE  DBK.DE     VOW3.DE     BMW.DE      ADS.DE      BEI.DE  \
2022-01-03  209.850006  11.348  180.800003  91.330002  259.100006   92.519997   
2022-01-04  215.850006  11.692  186.979996  94.190002  258.500000   91.120003   
2022-01-05  216.750000  11.850  191.000000  96.239998  260.850006   90.980003   
2022-01-06  220.000000  12.150  189.100006  96.220001  252.750000   89.680000   
2022-01-07  222.550003  12.366  187.740005  95.599998  247.449997   89.459999   
...                ...     ...         ...        ...         ...         ...   
2022-12-23  202.199997  10.706  115.660004  83.500000  125.860001  106.400002   
2022-12-27  203.050003  10.710  115.860001  83.650002  126.339996  107.300003   
2022-12-28  202.149994  10.630  114.959999  83.260002  125.779999  106.550003   
2022-12-29  203.699997  10.704  116.139999  83.830002  128.380005  107.449997   
2022-12-30  200.899994  10.588  116.419998  83.379997  127.459999  107.199997   

               DTE.SG      

# Mean-Variance Optimization

In [17]:
def mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # Start value of the weighs
    bounds = [(0, 1)] * n  # Border of the weighs (0-100% for every symbol)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, standard_deviations, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, standard_deviations, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x

In [18]:
def allocate_portfolio(historical_data, initial_capital, expected_returns_df, standard_deviations_df):
    # Calculate expected returns based on the predictions
    expected_returns = expected_returns_df['Average_Return'].values
    
    # Calculate standard deviations based on the predictions
    standard_deviations = standard_deviations_df['Standard_Deviation'].values
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
           
    print(covariance_matrix)
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation

In [19]:
initial_capital = 1000 # 1000€ start capital
portfolio_allocation = allocate_portfolio(combined_historical2018_data, initial_capital, average_return_per_symbol_df, standard_deviation_per_symbol_df)
print(portfolio_allocation)

             ALV.DE     DBK.DE     VOW3.DE      BMW.DE       ADS.DE  \
ALV.DE   378.761474  27.084220  247.738666  101.413813   583.230284   
DBK.DE    27.084220   2.431212   22.358885    8.756521    44.917376   
VOW3.DE  247.738666  22.358885  351.838272   99.039684   695.222668   
BMW.DE   101.413813   8.756521   99.039684   48.997457   168.208384   
ADS.DE   583.230284  44.917376  695.222668  168.208384  1855.978471   
BEI.DE   -60.246867  -5.015629  -73.746170  -14.842444  -190.363923   
DTE.SG   -11.497272  -0.888829  -13.654493   -2.836395   -36.326971   
SAP.DE   159.458635  11.849135  125.771314   59.439185   248.381242   
1COV.DE  135.265603  10.964016  125.933437   43.336186   287.687904   
BAS.DE   137.043177  11.655533  139.045799   50.372718   285.900562   
EOAN.DE   22.159195   1.813865   22.639072    7.174322    50.306592   
RWE.DE    -7.459152  -0.707160  -14.575938   -3.689588   -44.574248   

             BEI.DE     DTE.SG      SAP.DE     1COV.DE      BAS.DE    EOAN.D

In [20]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

7.978770172756013e-07
0.0001602934879008135
0.0001602934879008135
-----------------------------
9.785060765739962e-06
0.00010360422629916421
0.0002638977141999777
-----------------------------
1.2554200295520283e-06
0.00014615599754170443
0.00041005371174168215
-----------------------------
1.3643481632569338e-06
0.000113759346105069
0.0005238130578467512
-----------------------------
0.03768412779605245
4.803218894383996
4.8037427074418435
-----------------------------
3.1797059974766934e-05
0.0034086447322580894
4.807151352174102
-----------------------------
43.648090653874974
816.2193285283124
821.0264798804865
-----------------------------
1.1771563623845316e-06
0.00011346610105176578
821.0265933465876
-----------------------------
2.4377531709391516
89.09987653796792
910.1264698845555
-----------------------------
2.5662777480864245e-06
0.0001190496231673976
910.1265889341787
-----------------------------
1.0249007914849728e-05
9.566423612390856e-05
910.1266845984148
------------

In [21]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

241.9499969482422
7.978770172756013e-07
0.0001930463418949043
----------------------------
12.36400032043457
9.785060765739962e-06
0.0003140288363379849
----------------------------
111.8000030517578
1.2554200295520283e-06
0.00045438479947313955
----------------------------
100.77999877929688
1.3643481632569338e-06
0.0005918838057007092
----------------------------
184.16000366210935
0.03768412779605245
6.940500996730116
----------------------------
135.6999969482422
3.1797059974766934e-05
6.944815857671656
----------------------------
21.690000534057617
43.648090653874974
953.6719254508151
----------------------------
139.47999572753906
1.1771563623845316e-06
953.6720896405795
----------------------------
52.68000030517578
2.4377531709391516
1082.0929274295972
----------------------------
48.77999877929688
2.5662777480864245e-06
1082.0930526126226
----------------------------
12.149999618530272
1.0249007914849728e-05
1082.093177138065
----------------------------
41.18000030517578
2.1